In [17]:
import os
import pdfplumber
from collections import defaultdict

def detect_columns(text_blocks, page_width):
    """Gruppiere Textblöcke basierend auf ihrer horizontalen Position (Spalten)."""
    columns = defaultdict(list)
    threshold = page_width * 0.15  # Threshold, um Spalten zu erkennen

    for block in text_blocks:
        x_center = (block['x0'] + block['x1']) / 2

        assigned = False
        for col_x in columns:
            if abs(col_x - x_center) < threshold:
                columns[col_x].append(block)
                assigned = True
                break
        
        if not assigned:
            columns[x_center].append(block)

    return columns

def extract_text_from_column(column_blocks):
    """Sortiere die Blöcke innerhalb einer Spalte und extrahiere den Text."""
    sorted_blocks = sorted(column_blocks, key=lambda block: (block['top'], block['x0']))
    return ' '.join([block['text'] for block in sorted_blocks])

def extract_text_from_page_g1(page):
    """Extrahiere den Text aus einer Seite für PDFs in g1, behalte dabei das Layout."""
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""

    columns = detect_columns(text_blocks, page.width)
    full_text = []

    for col_x, blocks in sorted(columns.items()):
        column_text = extract_text_from_column(blocks)
        full_text.append(column_text)

    return "\n\n".join(full_text)

def extract_text_from_page_g2(page):
    """Alternative Methode zur Textextraktion für PDFs in g2."""
    # Beispielhafte einfache Extraktion, die besser auf das Layout abgestimmt ist
    # Dies kann durch Methoden wie pdfminer.six, die textorientierter sind, ergänzt werden.
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""
    
    full_text = [block['text'] for block in sorted(text_blocks, key=lambda block: (block['top'], block['x0']))]
    
    return ' '.join(full_text)

def extract_text_from_pdfs(pdf_folder_path, extraction_function):
    documents = []

    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder_path, filename)

            with pdfplumber.open(pdf_path) as pdf:
                full_text = []

                for page in pdf.pages:
                    page_text = extraction_function(page)
                    full_text.append(page_text)

                documents.append("\n\n".join(full_text))

    return documents

# Pfade zu den PDF-Ordnern
pdf_folder_path_g1 = os.path.join('data', 'pdf', 'g1')
pdf_folder_path_g2 = os.path.join('data', 'pdf', 'g2')

# Extrahiere die Texte aus den PDFs in g1
documents_g1 = extract_text_from_pdfs(pdf_folder_path_g1, extract_text_from_page_g1)

# Extrahiere die Texte aus den PDFs in g2
documents_g2 = extract_text_from_pdfs(pdf_folder_path_g2, extract_text_from_page_g2)

# Kombiniere alle Dokumente
documents = documents_g1 + documents_g2

# Beispielhafte Ausgabe des ersten Dokuments
# print(documents[2] if documents else "Keine Dokumente gefunden")
for i, doc in enumerate(documents):
    print(f"Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

Document 0:

Der blaue Altglascontainer. Das kommt hinein Saubere Flaschen und Deckelgläser nach den Farben Weiß, Grün und Braun getrennt; blaues oder buntes Glas bitte zum Grünglas geben.  Glasfl aschen  Konservengläser Das kommt nicht hinein  Flachglas wie Spiegel und Scheiben (Restmüll oder  Trinkgläser und Blumenvasen (Restmüll)  Porzellan und Keramik (Restmüll)  Glühbirnen (Restmüll), Energiesparlampen (Schadstoffmobil oder Wertstoffhöfe)  Stark verschmutzte Gläser (Restmüll) Standort: auf öffentlichen

immer Sperrmüll) Plätzen

Was kommt wo hinein? FES Frankfurter Entsorgungs- Abfall richtig trennen – und Service GmbH Weidenbornstraße 40 60389 Frankfurt am Main Servicetelefon 0800 2008007-0 Servicetelefax 069 212-31323 services@fes-frankfurt.de www.fes-frankfurt.de FES-Servicecenter Liebfrauenberg 52–54 (Ecke Töngesgasse/Schärfengässchen) 60313 Frankfurt am Main Mo. – Fr. 10.00 – 18.00 Uhr Sa. 10.00 – 16.00 Uhr rgungsfachbetrie Entso b rtschafte.V.-EdDn oE El -öK tsn 0 gla

In [12]:
import os
import pickle

def save_new_documents(documents, filename="documents.pkl"):
    # Sicherstellen, dass der Ordner, in dem die Datei gespeichert wird, existiert
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Speichern der Dokumente im angegebenen Pfad
    with open(filename, "wb") as file:
        pickle.dump(documents, file)
    
    print(f"Die Datei '{filename}' wurde erfolgreich gespeichert.")

# Beispielnutzung:
save_new_documents(documents, filename="data/pickle/documents.pkl")

Die Datei 'data/pickle/documents.pkl' wurde erfolgreich gespeichert.


In [18]:
import re

def normalize_text(documents):
    """Normalisiert die Dokumente und entfernt unnötige Zeichen."""
    normalized_documents = []

    for document in documents:
        text = document.lower()
        text = re.sub(r'\b(?:[\w\-]+\.)+[\w]{2,4}(?:/[\w\-.?%&=]*)?\b', '', text)
        text = re.sub(r'\b[\w\.\-]+@[\w\.\-]+\.\w{2,4}\b', '', text)
        text = re.sub(r'\b(?:www|servicetelefon|servicetelefax|services|frankfurt|main|weidenbornstraße|liebfrauenberg)\b', '', text)
        text = re.sub(r'[^a-zäöüß\s]', '', text)
        normalized_documents.append(text.strip())

    return normalized_documents

# Beispielnutzung:
normalized_documents = normalize_text(documents)
# Ausgabe der normalisierten Dokumente
for i, doc in enumerate(normalized_documents):
    print(f"Normalized Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

Normalized Document 0:

der blaue altglascontainer das kommt hinein saubere flaschen und deckelgläser nach den farben weiß grün und braun getrennt blaues oder buntes glas bitte zum grünglas geben  glasfl aschen  konservengläser das kommt nicht hinein  flachglas wie spiegel und scheiben restmüll oder  trinkgläser und blumenvasen restmüll  porzellan und keramik restmüll  glühbirnen restmüll energiesparlampen schadstoffmobil oder wertstoffhöfe  stark verschmutzte gläser restmüll standort auf öffentlichen

immer sperrmüll plätzen

was kommt wo hinein fes frankfurter entsorgungs abfall richtig trennen  und service gmbh     am          fesservicecenter   ecke töngesgasseschärfengässchen   am  mo  fr    uhr sa    uhr rgungsfachbetrie entso b rtschafteveddn oe el ök tsn  gla sn we rgergemeinschaftz de efür rrt dei nif di uk tma cht en s a m hae enl nn e elb ne vf tö sr od rae gr bn uf l eg r in u b se d o n ä l n n

richtig trennen leicht gemacht zu jedem haus in  gehören  abfallbehälter  tonn

In [19]:
import spacy

# Lade das deutsche Sprachmodell
nlp = spacy.load('de_core_news_sm')

# Wörterbuch zur Korrektur von Lemmata und POS-Tags
manual_corrections = {
    "deckelgläser": ("NOUN", "Deckelglas"),
    "konservengläser": ("NOUN", "Konservenglas"),
    "keramik": ("NOUN", "Keramik"),
    "glühbirnen": ("NOUN", "Glühbirne"),
    "energiesparlampen": ("NOUN", "Energiesparlampe"),
    "weiß": ("ADJ", "weiß"),
    "lacke": ("NOUN", "Lack"),
    "wertstoffhöfe": ("NOUN", "Wertstoffhof"),
}

def correct_token(token):
    if token.text.lower() in manual_corrections:
        return manual_corrections[token.text.lower()]
    else:
        lemma = token.lemma_
        pos = token.pos_
        return pos, lemma

def pos_tagging(documents):
    """Führt POS-Tagging und Lemmatisierung durch."""
    processed_documents = []

    for doc in documents:
        spacy_doc = nlp(doc)
        processed_tokens = []

        for token in spacy_doc:
            pos, lemma = correct_token(token)
            if pos in {"NOUN", "VERB", "ADJ"}:
                processed_tokens.append((token.text, pos, lemma))
        
        processed_documents.append(processed_tokens)

    return processed_documents

# Beispielnutzung:
pos_tagged_documents = pos_tagging(normalized_documents)

# Ausgabe der POS-getaggten Dokumente
for i, doc in enumerate(pos_tagged_documents):
    print(f"POS-Tagged Document {i}:\n")
    for token_info in doc:
        print(token_info)
    print("\n" + "="*50 + "\n")

POS-Tagged Document 0:

('blaue', 'ADJ', 'blau')
('altglascontainer', 'NOUN', 'Altglascontainer')
('kommt', 'VERB', 'kommen')
('saubere', 'ADJ', 'sauber')
('flaschen', 'NOUN', 'Flasche')
('deckelgläser', 'NOUN', 'Deckelglas')
('farben', 'NOUN', 'Farbe')
('weiß', 'ADJ', 'weiß')
('blaues', 'ADJ', 'blau')
('buntes', 'ADJ', 'bunt')
('glas', 'NOUN', 'Glas')
('grünglas', 'NOUN', 'Grünglas')
('geben', 'VERB', 'geben')
('aschen', 'ADJ', 'asch')
('konservengläser', 'NOUN', 'Konservenglas')
('kommt', 'VERB', 'kommen')
('scheiben', 'VERB', 'Scheiben')
('restmüll', 'NOUN', 'Restmüll')
('trinkgläser', 'NOUN', 'Trinkgläser')
('porzellan', 'NOUN', 'Porzellan')
('keramik', 'NOUN', 'Keramik')
('glühbirnen', 'NOUN', 'Glühbirne')
('restmüll', 'NOUN', 'Restmüll')
('energiesparlampen', 'NOUN', 'Energiesparlampe')
('wertstoffhöfe', 'NOUN', 'Wertstoffhof')
('verschmutzte', 'ADJ', 'verschmutzt')
('gläser', 'ADJ', 'Gläser')
('restmüll', 'ADJ', 'Restmüll')
('standort', 'NOUN', 'Standort')
('öffentlichen', 'ADJ'

In [22]:
import spacy

def remove_stopwords(processed_documents):
    """Entfernt unnötige Stopwörter aus den Dokumenten."""
    custom_stopwords = nlp.Defaults.stop_words.difference({'nicht', 'kein', 'wichtig', 'ohne', 'gehört', 'getrennt', 'hinein'})

    cleaned_documents = []

    for doc in processed_documents:
        filtered_tokens = [token_info for token_info in doc if token_info[2] not in custom_stopwords and len(token_info[2]) > 2]

        # Entferne unzusammenhängende Zeichenfolgen und extrem kurze Wörter
        cleaned_text = ' '.join([token_info[2] for token_info in filtered_tokens])
        cleaned_text = re.sub(r'\b\w{1,3}\b', '', cleaned_text)

        # Entferne mehrfach aufeinanderfolgende gleiche Wörter
        cleaned_text = re.sub(r'\b(\w+)\s+\1\b', r'\1', cleaned_text)

        # Entferne weiterhin sinnlose Textfragmente
        cleaned_text = re.sub(r'\b[a-zäöüß]{2,3}\b\s+\b[a-zäöüß]{2,3}\b', '', cleaned_text)

        cleaned_documents.append(cleaned_text.strip())

    return cleaned_documents

# Verwende die POS-getaggten Dokumente als Input für das Entfernen von Stopwörtern
cleaned_documents = remove_stopwords(pos_tagged_documents)

# Ausgabe der bereinigten Dokumente
for i, doc in enumerate(cleaned_documents):
    print(f"Cleaned Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

Cleaned Document 0:

blau Altglascontainer sauber Flasche Deckelglas Farbe weiß blau bunt Glas Grünglas geben asch Konservenglas Scheiben Restmüll Trinkgläser Porzellan Keramik Glühbirne Restmüll Energiesparlampe Wertstoffhof verschmutzt Gläser Restmüll Standort öffentlich Sperrmüll plätzen Frankfurter Entsorgung Abfall trennen Service Gmbh Fesservicecenter Ecke töngesgasseschärfengässchen   Rgungsfachbetrie rren trennen Haus gehören Abfallbehälter Tonne Inhalt Wertvoller Rohstoff  Produkt weiterverarbeiten grau Restmülltonne gelb Tonne verpackunger grün Tonne Papier Pappe braun Tonne bioabfälle grau Tonne restlich Abfall Restmüll geben folgend entsorgungsangebot wertstoffe sichern Altglascontainer Flasche Deckelglas nden öffentlich Plätz altkleid Container altkleider schuhen nden Stadtteil öffentlich Plätz Feswertstoffhöf Schadstoff Abfäll gefährlich Stoff farbe Spraydos mitteln Fesschadstoffmobil abgeben terminen nden Jahr Haushalt verteilen Sperrmüllabholung holen  Termin Wohnsiedlu

In [23]:
import re
import spacy

def remove_stopwords_and_clean(processed_documents):
    """Entfernt Stopwörter und bereinigt die Token-Informationen weiter."""
    custom_stopwords = nlp.Defaults.stop_words.difference({'nicht', 'kein', 'wichtig', 'ohne'})
    
    cleaned_documents = []

    for doc in processed_documents:
        # Filtere die Tokens, indem du Stopwörter entfernst und unnötige Zeichenbereinigungen durchführst
        filtered_tokens = []
        for token_info in doc:
            lemma = token_info[2]
            if lemma not in custom_stopwords and len(lemma) > 2:
                cleaned_lemma = re.sub(r'\b\w{1,3}\b', '', lemma)
                cleaned_lemma = re.sub(r'\b(\w+)\s+\1\b', r'\1', cleaned_lemma)
                cleaned_lemma = re.sub(r'\b[a-zäöüß]{2,3}\b\s+\b[a-zäöüß]{2,3}\b', '', cleaned_lemma)
                filtered_tokens.append((token_info[0], token_info[1], cleaned_lemma))
        
        cleaned_documents.append(filtered_tokens)

    return cleaned_documents

# Verwende die POS-getaggten Dokumente als Input für das Entfernen von Stopwörtern
cleaned_documents = remove_stopwords_and_clean(pos_tagged_documents)

# Ausgabe der bereinigten Dokumente mit den Token-Informationen
for i, doc in enumerate(cleaned_documents):
    print(f"Final Token Information for Document {i}:\n")
    for token_info in doc:
        print(f"Token: {token_info[0]}, POS: {token_info[1]}, Lemma: {token_info[2]}")
    print("\n" + "="*50 + "\n")

Final Token Information for Document 0:

Token: blaue, POS: ADJ, Lemma: blau
Token: altglascontainer, POS: NOUN, Lemma: Altglascontainer
Token: saubere, POS: ADJ, Lemma: sauber
Token: flaschen, POS: NOUN, Lemma: Flasche
Token: deckelgläser, POS: NOUN, Lemma: Deckelglas
Token: farben, POS: NOUN, Lemma: Farbe
Token: weiß, POS: ADJ, Lemma: weiß
Token: blaues, POS: ADJ, Lemma: blau
Token: buntes, POS: ADJ, Lemma: bunt
Token: glas, POS: NOUN, Lemma: Glas
Token: grünglas, POS: NOUN, Lemma: Grünglas
Token: geben, POS: VERB, Lemma: geben
Token: aschen, POS: ADJ, Lemma: asch
Token: konservengläser, POS: NOUN, Lemma: Konservenglas
Token: scheiben, POS: VERB, Lemma: Scheiben
Token: restmüll, POS: NOUN, Lemma: Restmüll
Token: trinkgläser, POS: NOUN, Lemma: Trinkgläser
Token: porzellan, POS: NOUN, Lemma: Porzellan
Token: keramik, POS: NOUN, Lemma: Keramik
Token: glühbirnen, POS: NOUN, Lemma: Glühbirne
Token: restmüll, POS: NOUN, Lemma: Restmüll
Token: energiesparlampen, POS: NOUN, Lemma: Energiespa

In [24]:
# Ausgabe der bereinigten Dokumente mit den Token-Informationen
for i, doc in enumerate(cleaned_documents):
    print(f"Final Text for Document {i}:\n")
    final_text = " ".join([token_info[0] for token_info in doc])
    print(final_text)
    
    print(f"\nFinal Token Information for Document {i}:\n")
    for token_info in doc:
        print(f"Token: {token_info[0]}, POS: {token_info[1]}, Lemma: {token_info[2]}")
    print("\n" + "="*50 + "\n")

Final Text for Document 0:

blaue altglascontainer saubere flaschen deckelgläser farben weiß blaues buntes glas grünglas geben aschen konservengläser scheiben restmüll trinkgläser porzellan keramik glühbirnen restmüll energiesparlampen wertstoffhöfe verschmutzte gläser restmüll standort öffentlichen sperrmüll plätzen frankfurter entsorgungs abfall trennen service gmbh fesservicecenter ecke töngesgasseschärfengässchen uhr uhr rgungsfachbetrie rrt trennen haus gehören abfallbehälter tonnen inhalt wertvoller rohstoff neuen produkten weiterverarbeitet graue restmülltonne gelbe tonne verpackungen grüne tonne papier pappe braune tonne bioabfälle graue tonne restlichen abfall restmüll gibt folgende entsorgungsangebote wertstoffe sichern altglascontainer flaschen deckelgläser nden öffentlichen plätzen altkleider container altkleider schuhe nden stadtteil öffentlichen plätzen feswertstoffhöfen schadstoffe abfälle gefährlichen stoffen farben spraydosen mitteln fesschadstoffmobil abgeben termine 

In [7]:
import os
import pickle

def save_new_documents(documents, filename="new_processed_documents.pkl"):
    # Sicherstellen, dass der Ordner, in dem die Datei gespeichert wird, existiert
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Speichern der Dokumente im angegebenen Pfad
    with open(filename, "wb") as file:
        pickle.dump(documents, file)
    
    print(f"Die Datei '{filename}' wurde erfolgreich gespeichert.")

# Beispielnutzung:
save_new_documents(cleaned_documents, filename="data/pickle/new_processed_documents.pkl")

Die Datei 'data/pickle/new_processed_documents.pkl' wurde erfolgreich gespeichert.


In [25]:
import re

def chunk_documents_with_context(cleaned_documents, max_chunk_size=100):
    """
    Teilt Dokumente in sinnvolle Chunks, um den Kontext nicht zu verlieren und druckt die Chunks.
    
    Args:
    - cleaned_documents: Eine Liste von Listen, wobei jede innere Liste ein Dokument darstellt.
    - max_chunk_size: Maximale Anzahl von Tokens pro Chunk.
    
    Returns:
    - Eine Liste von Chunks.
    """
    chunks = []
    for i, document in enumerate(cleaned_documents):
        print(f"\nDokument {i+1}:")
        current_chunk = []
        current_size = 0

        for j, (token, pos, lemma) in enumerate(document):
            current_chunk.append((token, pos, lemma))
            current_size += 1

            # Kontextlogik für die Trennung von Chunks
            if current_size >= max_chunk_size:
                # Überprüfe, ob der nächste Token ein Noun (NOUN) oder ein Adjektiv (ADJ) ist,
                # um den Zusammenhang zu erhalten
                if j+1 < len(document) and document[j+1][1] in {'NOUN', 'ADJ'}:
                    continue  # Warte mit dem Chunking, um den Zusammenhang nicht zu trennen
                
                # Füge das aktuelle Chunk der Chunks-Liste hinzu
                chunks.append(current_chunk)
                
                # Print das aktuelle Chunk
                print("Chunk:", current_chunk)
                
                # Setze das aktuelle Chunk und die Größe zurück
                current_chunk = []
                current_size = 0
        
        # Füge das letzte Chunk hinzu, wenn es nicht leer ist
        if current_chunk:
            chunks.append(current_chunk)
            print("Chunk:", current_chunk)

    return chunks

# Beispielaufruf
# cleaned_documents ist deine Liste von Dokumenten
chunks = chunk_documents_with_context(cleaned_documents, max_chunk_size=100)


Dokument 1:
Chunk: [('blaue', 'ADJ', 'blau'), ('altglascontainer', 'NOUN', 'Altglascontainer'), ('saubere', 'ADJ', 'sauber'), ('flaschen', 'NOUN', 'Flasche'), ('deckelgläser', 'NOUN', 'Deckelglas'), ('farben', 'NOUN', 'Farbe'), ('weiß', 'ADJ', 'weiß'), ('blaues', 'ADJ', 'blau'), ('buntes', 'ADJ', 'bunt'), ('glas', 'NOUN', 'Glas'), ('grünglas', 'NOUN', 'Grünglas'), ('geben', 'VERB', 'geben'), ('aschen', 'ADJ', 'asch'), ('konservengläser', 'NOUN', 'Konservenglas'), ('scheiben', 'VERB', 'Scheiben'), ('restmüll', 'NOUN', 'Restmüll'), ('trinkgläser', 'NOUN', 'Trinkgläser'), ('porzellan', 'NOUN', 'Porzellan'), ('keramik', 'NOUN', 'Keramik'), ('glühbirnen', 'NOUN', 'Glühbirne'), ('restmüll', 'NOUN', 'Restmüll'), ('energiesparlampen', 'NOUN', 'Energiesparlampe'), ('wertstoffhöfe', 'NOUN', 'Wertstoffhof'), ('verschmutzte', 'ADJ', 'verschmutzt'), ('gläser', 'ADJ', 'Gläser'), ('restmüll', 'ADJ', 'Restmüll'), ('standort', 'NOUN', 'Standort'), ('öffentlichen', 'ADJ', 'öffentlich'), ('sperrmüll', '

In [26]:
import os
import pickle

def save_new_documents(documents, filename="chunks.pkl"):
    # Sicherstellen, dass der Ordner, in dem die Datei gespeichert wird, existiert
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Speichern der Dokumente im angegebenen Pfad
    with open(filename, "wb") as file:
        pickle.dump(documents, file)
    
    print(f"Die Datei '{filename}' wurde erfolgreich gespeichert.")

# Beispielnutzung:
save_new_documents(chunks, filename="data/pickle/chunks.pkl")

Die Datei 'data/pickle/chunks.pkl' wurde erfolgreich gespeichert.
